In [1]:
from datasets import load_dataset



# Load the dataset
dataset = load_dataset("conll2003", trust_remote_code=True)

train_dataset = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']



In [2]:
print(train_dataset)

Dataset({
    features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})


In [3]:
labels = dataset["train"].features["ner_tags"].feature.names
num_labels = len(labels)
print(labels)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
from transformers import AutoModelForTokenClassification

model_checkpoint = "bert-base-cased"

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit
    data = data[:nbatch * bsz]
    # Evenly divide the data across the bsz batches
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

In [7]:
# batch_size = 20
# eval_batch_size = 10
# 
# 
# 
# # train_data = batchify(train_dataset, batch_size)
# # val_data = batchify(valid_dataset, eval_batch_size)
# # test_data = batchify(test_dataset, eval_batch_size)
# 
# test = list(train_dataset.values())
# print(test)

In [8]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained("bert-base-cased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)


In [9]:
print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.6023,  0.1092,  0.1417,  ..., -0.4177,  0.6059,  0.1764],
         [ 0.5119, -0.4770,  0.5508,  ..., -0.2814,  0.3793,  0.1156],
         [ 0.0995,  0.0867,  0.0869,  ...,  0.4789, -0.3236,  0.3122],
         ...,
         [ 0.8081, -0.7380,  0.2001,  ...,  0.7405, -0.7998,  0.6449],
         [ 0.3305, -0.1958,  0.3148,  ..., -0.0525,  0.5358,  0.1987],
         [ 0.5655, -0.2176, -0.4720,  ..., -0.3554,  0.6141, -0.2476]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-8.2835e-01,  6.0906e-01,  9.9998e-01, -9.9831e-01,  9.8739e-01,
          9.2930e-01,  9.9664e-01, -9.8909e-01, -9.9124e-01, -7.8317e-01,
          9.9476e-01,  9.9961e-01, -9.9850e-01, -9.9997e-01,  8.0649e-01,
         -9.9098e-01,  9.9545e-01, -6.9968e-01, -9.9999e-01, -2.6556e-01,
         -4.1867e-01, -9.9998e-01,  3.1562e-01,  9.7638e-01,  9.9257e-01,
          1.4021e-01,  9.9543e-01,  9.9999e-01,  9.4214e-01, -2.024

In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


[{'entity': 'LABEL_1', 'score': np.float32(0.5220458), 'index': 1, 'word': 'my', 'start': 0, 'end': 2}, {'entity': 'LABEL_0', 'score': np.float32(0.7009999), 'index': 2, 'word': 'name', 'start': 3, 'end': 7}, {'entity': 'LABEL_0', 'score': np.float32(0.6081854), 'index': 3, 'word': 'is', 'start': 8, 'end': 10}, {'entity': 'LABEL_0', 'score': np.float32(0.5473961), 'index': 4, 'word': 'wolfgang', 'start': 11, 'end': 19}, {'entity': 'LABEL_0', 'score': np.float32(0.6817879), 'index': 5, 'word': 'and', 'start': 20, 'end': 23}, {'entity': 'LABEL_0', 'score': np.float32(0.55994314), 'index': 6, 'word': 'i', 'start': 24, 'end': 25}, {'entity': 'LABEL_0', 'score': np.float32(0.61987853), 'index': 7, 'word': 'live', 'start': 26, 'end': 30}, {'entity': 'LABEL_0', 'score': np.float32(0.7031592), 'index': 8, 'word': 'in', 'start': 31, 'end': 33}, {'entity': 'LABEL_1', 'score': np.float32(0.5161929), 'index': 9, 'word': 'berlin', 'start': 34, 'end': 40}]


In [11]:
def custom_tokenize(text):
    return [token + ' ' for token in text.split()]


model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
count = 0
total = len(test_dataset)
for data in test_dataset:
    seq = " ".join([token for token in data['tokens']])
    ner_results = nlp(seq)

    for i in range(len(ner_results)):
        # print(ner_results[i]['entity'])
        # print(data)
        
        if ner_results[i]['entity'] == data['ner_tags'][i]:
            count += 1

    

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.']
['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.']
['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.']
['Japan', 'began', 'the', 'defence', 'of', 'their', 'Asian', 'Cup', 'title', 'with', 'a', 'lucky', '2-1', 'win', 'against', 'Syria', 'in', 'a', 'Group', 'C', 'championship', 'match', 'on', 'Friday', '.']
['Japan', 'began', 'the', 'defence', 'of', 'their', 'Asian', 'Cup', 'title', 'with', 'a', 'lucky', '2-1', 'win', 'against', 'Syria', 'in', 'a', 'Group', 'C', 'championship', 'match', 'on', 'Friday', '.']
['Japan', 'began', 'the', 'defence', 'of', 'their', 'Asian', 'Cup', 'title', 'with', 'a', 'lucky', '2-1', 'win', 'against', 'Syria', 'in', 'a', 'Group', 'C', 'championship', 'match', 'on', 'Friday', '.']
['Japan', 'began', 'the', 'defence', 'of', 'their', 'Asian', 'Cup', 'title', 'with', 'a', 'lucky', '2-1

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
print(count/total)

In [ ]:

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input text
text = "SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT ."

# Tokenize the text
tokens = tokenizer.tokenize(text)

# Printing the tokens
print(tokens)